In [1]:
import os 
import warnings
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
warnings.filterwarnings('ignore')

In [120]:
gas = pd.read_csv('tidy_gas.csv')

In [126]:
gas['gas'] = pd.to_numeric(gas['gas'], errors='coerce')

cumulative_df = gas.groupby('country', as_index=False)['gas'].sum()
cumulative_df.rename(columns={'gas': 'Cumulative production'}, inplace=True)
cumulative_df['Cumulative production'].fillna(0, inplace=True)

cumulative_df['country'] = cumulative_df['country'].str.title()
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Cumulative Natural Gas Production by Country, 1900-2022"),
    dcc.Graph(id='map-graph', style={'height': '80vh'}),

    html.Div([
        dcc.Input(id='country-input', type='text', placeholder='Enter a country name...'),
        html.Button('Search', id='search-button', n_clicks=0)
    ]),
    html.Div(id='country-output')
])

@app.callback(
    [Output('map-graph', 'figure'),
     Output('country-output', 'children')],
    [Input('search-button', 'n_clicks')],
    [dash.dependencies.State('country-input', 'value')]
)
def update_map(n_clicks, country_name):
    fig = px.choropleth(
        cumulative_df,
        locations="country",
        locationmode="country names",
        color="Cumulative production",
        hover_name="country",
        color_continuous_scale=px.colors.sequential.Reds,
        range_color=(0, 1468), 
        title="Cumulative Natural Gas Production by Country, 1900-2022"
    )

    fig.update_geos(
        visible=False,
        resolution=50,
        showcountries=True,
        countrycolor="Black"
    )
    fig.update_layout(
        margin={"r":0,"t":50,"l":0,"b":0},  
        coloraxis_colorbar=dict(
            title="Cumulative production (EJ)",
            tickvals=[0, 200, 500, 1000, 1468], 
            ticks="outside"
        )
    )

    if country_name:
        country_data = cumulative_df[cumulative_df['country'].str.contains(country_name, case=False, na=False)]
        if not country_data.empty:
            output_message = f"{country_name.title()}: Cumulative production = {country_data.iloc[0]['Cumulative production']:.2f} EJ"
        else:
            output_message = "Country not found"
    else:
        output_message = ""

    return fig, output_message

if __name__ == '__main__':
    app.run_server(debug=True)


In [127]:
file_path = 'gas.csv'
df = pd.read_csv(file_path)

df_long = df.melt(id_vars=['Country'], var_name='Year', value_name='Gas Production')

df_long['Year'] = pd.to_numeric(df_long['Year'], errors='coerce')

df_long = df_long[df_long['Gas Production'].notna()]

top10_data = df_long.groupby('Year', group_keys=False).apply(lambda x: x.nlargest(10, 'Gas Production')).reset_index(drop=True)


fig = px.bar(top10_data,
             x='Gas Production',
             y='Country',
             color='Country',
             animation_frame='Year',
             animation_group='Country',
             orientation='h',
             title='Top 10 Natural Gas Producing Countries by Year',
             labels={'Gas Production': 'Natural Gas Production (EJ)', 'Country': 'Country'},
             text_auto=True 
)

year_max_values = top10_data.groupby('Year')['Gas Production'].max()

x_max = year_max_values.max() * 1.1

fig.update_layout(
    xaxis_title='Natural Gas Production (EJ)',
    yaxis_title=' ',
    template='plotly_white',
    height=600,
    width=1000,
    yaxis={
        'categoryorder': 'total ascending',
        'automargin': False, 
    },
    xaxis=dict(
        range=[0, x_max],
    ),
    bargap=0.1,
    updatemenus=[{
        'type': 'buttons',
        'buttons': [
            {
                'label': 'Play',
                'method': 'animate',
                'args': [None, {
                    'frame': {'duration': 800, 'redraw': True},
                    'transition': {'duration': 600, 'easing': 'easeInOutQuad'},
                    'fromcurrent': True
                }]
            },
            {
                'label': 'Pause',
                'method': 'animate',
                'args': [[None], {'frame': {'duration': 0}, 'mode': 'immediate'}]
            }
        ]
    }]
)

fig.update_layout(
    sliders=[{
        'steps': [
            {
                'label': str(year),
                'method': 'animate',
                'args': [[str(year)], {
                    'mode': 'immediate',
                    'frame': {'duration': 800, 'redraw': True},
                    'transition': {'duration': 600, 'easing': 'easeInOutQuad'}
                }]
            }
            for year in top10_data['Year'].unique()
        ],
        'currentvalue': {
            'font': {'size': 20},
            'prefix': 'Year: ',
            'visible': True,
            'xanchor': 'right'
        },
    }]
)

for frame in fig.frames:
    year = int(frame.name)
    max_value_for_year = year_max_values.loc[year]
    total_value = top10_data[top10_data['Year'] == year]['Gas Production'].sum()

    frame.layout = go.Layout(
        xaxis=dict(range=[0, max_value_for_year * 1.1], autorange=False),
        annotations=[
            go.layout.Annotation(
                text=f"<b>{year}</b>",
                xref="paper",
                yref="paper",
                x=1,
                y=0.05,
                showarrow=False,
                font=dict(size=36, color="grey")
            ),
            go.layout.Annotation(
                text=f"Total: {total_value:.2f}",
                xref="paper",
                yref="paper",
                x=1,
                y=0.01,
                showarrow=False,
                font=dict(size=20, color="lightgrey")
            )
        ]
    )

fig.show()
